This can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2025.1_tutorials/wallaroo-ml-operations/model-automation/orchestration_sdk_run_continuously_tutorial).

## Wallaroo ML Automation Run Continuously Task Tutorial

This tutorial provides a tutorial for the Run Continuously Task for ML Orchestrations.

## Definitions

* **Orchestration**: A set of instructions written as a python script with a requirements library.  Orchestrations are uploaded to the Wallaroo instance as a .zip file.
* **Task**: An implementation of an orchestration.  Tasks are run either once when requested, on a repeating schedule, or continuously.
* **Connection**: Definitions set by MLOps engineers that are used by other Wallaroo users for connection information to a data source.  Usually paired with orchestrations.

Tasks are created from an orchestration through the following methods.

| Task Type | Orchestration Method | Description |
|---|---|---|
| [Run Once](#run-task-once) | `run_once` | Create one Task Run and end when the Task Run is finished executing or until the `timeout` for the task is reached . |
| [Run Scheduled](#run-task-scheduled) | `run_scheduled` | Based on a schedule, create a new Task Run until the Scheduled Task is terminated.  Repeat every time the schedule pattern is fulfilled (every hour, every Tuesday at 2 PM, etc).  Continue generating new Task Runs until the Run Scheduled Task is terminated. |
| [Run Continuously](#run-task-continuously) | `run_continuously` | Generate a Task Run and continue running that task.  These tasks typically have a repeating loop that continues to run until the Run Continuously Task is issued the `kill` command. |


## Tutorial Goals

The tutorial will demonstrate the following:

1. Create a Wallaroo connection to retrieving information from an external source.
1. Upload Wallaroo ML Workload Orchestration with a task meant for Run Continuously Tasks.  The instructions are:
  1. Deploy a previously defined pipeline.
  1. Every 1 minute, download data through the defined connection.  This simulates using a database connection to retrieve the latest data from a data store.
  1. Perform an inference with the downloaded data.
1. Create a Run Continuously Task and verify that the loop has pulled data and performed the inference request.
1. Kill the task and undeploy the pipeline used to end the tutorial.


## Prerequisites

* An installed Wallaroo instance.
* The following Python libraries installed.  These are included by default in a Wallaroo instance's JupyterHub service.
  * `os`
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.
  * [`pandas`](https://pypi.org/project/pandas/): Pandas, mainly used for Pandas DataFrame
  * [`pyarrow`](https://pypi.org/project/pyarrow/): PyArrow for Apache Arrow support

## Orchestration Setup Steps

For this tutorial, we'll create a workspace, upload our sample model and deploy a pipeline.  We'll perform some quick sample inferences to verify that everything it working.

### Load Libraries

Here we'll import the various libraries we'll use for the tutorial.

In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError, RequiredAttributeMissing

# to display dataframe tables
from IPython.display import display
# used to display dataframe information without truncating
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
import pyarrow as pa

import time

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [3]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

In [27]:
# Setting variables for later steps

workspace_name = f'continuoustutorial'
pipeline_name = f'continuouspipeline'
model_name = f'houseprice'
model_file_name = './models/rf_model.onnx'

connection_name = f'continuous_data_sample'
connection_type = "HTTP"
connection_argument = {'host':'https://github.com/WallarooLabs/Wallaroo_Tutorials/raw/refs/heads/wallaroo2025.1_tutorials/wallaroo-automate/orchestration_sdk_simple_tutorial/data/xtest-1k.arrow'}

### Create the Workspace and Pipeline

We'll now create our workspace and pipeline for the tutorial.  If this tutorial has been run previously, then this will retrieve the existing ones with the assumption they're for us with this tutorial.

We'll set the retrieved workspace as the current workspace in the SDK, so all commands will default to that workspace.

In [5]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)
wl.set_current_workspace(workspace)

pipeline = wl.build_pipeline(pipeline_name)

### Upload the Model and Deploy Pipeline

We'll upload our model into our sample workspace, then add it as a pipeline step before deploying the pipeline to it's ready to accept inference requests.

In [6]:
# Upload the model

housing_model_control = (wl.upload_model(model_name, 
                                         model_file_name, 
                                         framework=wallaroo.framework.Framework.ONNX)
                                         .configure(tensor_fields=["tensor"])
                        )

# Add the model as a pipeline step

pipeline.add_model_step(housing_model_control)

name,continuouspipeline
created,2024-09-24 17:11:25.717188+00:00
last_updated,2024-09-24 17:11:25.717188+00:00
deployed,(none)
workspace_id,9
workspace_name,continuoustutorial
arch,None
accel,None
tags,
versions,8ae1c528-d714-4060-af99-98dbf0a93dd0
steps,


In [11]:
#deploy the pipeline
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()

pipeline.deploy(deployment_config=deploy_config, wait_for_status=False)

Deployment initiated for continuouspipeline. Please check pipeline status.


name,continuouspipeline
created,2024-09-24 17:11:25.717188+00:00
last_updated,2024-09-24 17:15:12.541464+00:00
deployed,True
workspace_id,9
workspace_name,continuoustutorial
arch,x86
accel,none
tags,
versions,"759bd273-6b20-4cc2-87c1-7c48b81686f4, 085f7dd4-72fb-4a16-928b-bf3fb71b0baa, 8ae1c528-d714-4060-af99-98dbf0a93dd0"
steps,houseprice


In [12]:
import time

while pipeline.status()['status'] != 'Running':
    time.sleep(15)
    print("Waiting for deployment.")

pipeline.status()

Waiting for deployment.


{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.28.2.54',
   'name': 'engine-df98f4788-q9jjw',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'continuouspipeline',
      'status': 'Running',
      'version': '759bd273-6b20-4cc2-87c1-7c48b81686f4'}]},
   'model_statuses': {'models': [{'model_version_id': 7,
      'name': 'houseprice',
      'sha': 'e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6',
      'status': 'Running',
      'version': '5b7438ae-04f0-4398-8dae-3d7a0370e746'}]}}],
 'engine_lbs': [{'ip': '10.28.2.53',
   'name': 'engine-lb-6b59985857-j7475',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

## Create Connections

We will create the data source connection via the Wallaroo client command `create_connection`.

Connections are created with the Wallaroo client command [`create_connection`](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-ml-workload-orchestration/#create-orchestration) with the following parameters.

| Parameter | Type | Description |
| --- | --- | ---|
| **name** | string (Required) | The name of the connection. This must be unique - **if submitting the name of an existing** connection it will return an error. |
| **type** | string (Required) | The user defined type of connection. |
| **details** | Dict (Required) | User defined configuration details for the data connection.  These can be `{'username':'dataperson', 'password':'datapassword', 'port': 3339}`, or `{'token':'abcde123==', 'host':'example.com', 'port:1234'}`, or other user defined combinations.  |

* **IMPORTANT NOTE**:  Data connections names **must** be unique.  Attempting to create a data connection with the same `name` as an existing data connection will result in an error.

We'll also create a data connection named `inference_results_connection` with our helper function `get_connection` that will either create **or** retrieve a connection if it already exists.  From there we'll create out connections:  

* `houseprice_arrow_table`: An Apache Arrow file stored on GitHub that will be used for our inference input.

In [28]:
wl.create_connection(connection_name, connection_type, connection_argument)

Field,Value
Name,continuous_data_sample
Connection Type,HTTP
Details,*****
Created At,2024-09-24T17:29:30.959757+00:00
Linked Workspaces,[]


### Test Orchestration Script

To verify the orchestration will work, we'll test the relevant code here.  The complete code is in `./remote_inference/main.py`.

The run below will run through the inference loop twice to verify the loop code works.

In [68]:
print("Deploy the pipeline.")
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()

pipeline.deploy(deployment_config=deploy_config, wait_for_status=False)

# verify the pipeline is running

while pipeline.status()['status'] != 'Running':
    time.sleep(15)
    print("Waiting for deployment.")

print(pipeline.status())

# Get the connection - assuming it will be the only one

inference_source_connection = wl.get_connection(name=connection_name)

# our continuous loop - check every minute for the file.
# in a real example, this would be a database with a search filter to inference on the latest
# updates, then store the inference results in another data store

# for this test, we'll just run through twice to verify it works
for x in range (2):
    time.sleep(60)

    print(f"Getting arrow table file")
    # Retrieve the file
    # set accept as apache arrow table
    headers = {
        'Accept': 'application/vnd.apache.arrow.file'
    }

    response = requests.get(
                        inference_source_connection.details()['host'], 
                        headers=headers
                    )

    # Arrow table is retrieved 
    with pa.ipc.open_file(response.content) as reader:
        arrow_table = reader.read_all()

    print("Inference time.  Displaying results after.")
    # Perform the inference
    result = pipeline.infer(arrow_table)

    result_dataframe = result.to_pandas()

    print(result_dataframe.head(5))


Deploy the pipeline.
Deployment initiated for continuouspipeline. Please check pipeline status.
{'status': 'Running', 'details': [], 'engines': [{'ip': '10.28.2.56', 'name': 'engine-8595d79f94-8zvrs', 'status': 'Running', 'reason': None, 'details': [], 'pipeline_statuses': {'pipelines': [{'id': 'continuouspipeline', 'status': 'Running', 'version': 'fd88bf40-1305-491f-9c93-7c27fc49e754'}]}, 'model_statuses': {'models': [{'model_version_id': 7, 'name': 'houseprice', 'sha': 'e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6', 'status': 'Running', 'version': '5b7438ae-04f0-4398-8dae-3d7a0370e746'}]}}], 'engine_lbs': [{'ip': '10.28.2.55', 'name': 'engine-lb-6b59985857-jwwtb', 'status': 'Running', 'reason': None, 'details': []}], 'sidekicks': []}
Getting arrow table file
Inference time.  Displaying results after.
                     time  \
0 2024-09-24 17:58:40.884   
1 2024-09-24 17:58:40.884   
2 2024-09-24 17:58:40.884   
3 2024-09-24 17:58:40.884   
4 2024-09-24 17:58:40

Before we continue to the Run Continuously Task example, we'll undeploy the pipeline since our Orchestration deploys the pipeline as part of the task cycle.

In [69]:
pipeline.undeploy()

Waiting for undeployment - this will take up to 45s ................................... ok


name,continuouspipeline
created,2024-09-24 17:11:25.717188+00:00
last_updated,2024-09-24 17:57:40.189304+00:00
deployed,False
workspace_id,9
workspace_name,continuoustutorial
arch,x86
accel,none
tags,
versions,"ba43237a-94ca-4375-a259-484775e9ed46, fd88bf40-1305-491f-9c93-7c27fc49e754, bf8e8426-d3ae-4f8d-a549-3ebd7bb30ae8, 6be2d1d9-719f-42ff-8cd6-d253ad924b7f, 759bd273-6b20-4cc2-87c1-7c48b81686f4, 085f7dd4-72fb-4a16-928b-bf3fb71b0baa, 8ae1c528-d714-4060-af99-98dbf0a93dd0"
steps,houseprice


## Wallaroo ML Workload Run Continuously Task Example

With the pipeline deployed and our connections set, we will now generate our ML Workload Orchestration.  See the [Wallaroo ML Workload Orchestrations guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-ml-workload-orchestration/) for full details.

Orchestrations are uploaded to the Wallaroo instance as a ZIP file with the following requirements:

| Parameter | Type | Description |
|---|---|---|
| **User Code** | (*Required*) Python script as `.py` files | If `main.py` exists, then that will be used as the task entrypoint. Otherwise, the **first** `main.py` found in any subdirectory will be used as the entrypoint. |
| **Python Library Requirements** | (*Optional*) `requirements.txt` file in the [requirements file format](https://pip.pypa.io/en/stable/reference/requirements-file-format/).  A standard Python requirements.txt for any dependencies to be provided in the task environment. The Wallaroo SDK will already be present and **should not be included in the requirements.txt**. Multiple requirements.txt files are not allowed. |
| **Other artifacts** | &nbsp; | Other artifacts such as files, data, or code to support the orchestration.

For our example, our orchestration will:


1. Upload Wallaroo ML Workload Orchestration with a task meant for Run Continuously Tasks.  The instructions are:
  1. Deploy a previously defined pipeline.
  1. Every 1 minute, download data through the defined connection.  This simulates using a database connection to retrieve the latest data from a data store.
  1. Perform an inference with the downloaded data.

This sample script is stored in `remote_inference/main.py` with an empty `requirements.txt` file, and packaged into the orchestration as `./remote_inference/remote_inference.zip`.  We'll display the steps in uploading the orchestration to the Wallaroo instance.

### Upload the Orchestration

Orchestrations are uploaded with the Wallaroo client `upload_orchestration(path)` method with the following parameters.

| Parameter | Type | Description |
| --- | --- | ---|
| **path** | string (Required) | The path to the ZIP file to be uploaded. |

Once uploaded, the deployment will be prepared and any requirements will be downloaded and installed.


For this example, the orchestration `./remote_inference/remote_inference.zip` will be uploaded and saved to the variable `orchestration`.

### Orchestration Status

We will loop until the uploaded orchestration's `status` displays `ready`.

In [168]:
orchestration = wl.upload_orchestration(path="./remote_inference/remote_inference.zip")

while orchestration.status() != 'ready' and orchestration.status() != 'error':
    print(orchestration.status())
    time.sleep(5)

pending_packaging
pending_packaging
packaging
packaging
packaging
packaging
packaging
packaging
packaging
packaging


## Task Management Tutorial

Once an Orchestration has the status `ready`, it can be run as a Task.  

Tasks are created from an Orchestration through the following methods.

| Task Type | Orchestration Method | Description |
|---|---|---|
| [Run Once](#run-task-once) | `run_once` | Create one Task Run and end when the Task Run is finished executing or until the `timeout` for the task is reached . |
| [Run Scheduled](#run-task-scheduled) | `run_scheduled` | Based on a schedule, create a new Task Run until the Scheduled Task is terminated.  Repeat every time the schedule pattern is fulfilled (every hour, every Tuesday at 2 PM, etc).  Continue generating new Task Runs until the Run Scheduled Task is terminated. |
| [Run Continuously](#run-task-continuously) | `run_continuously` | Generate a Task Run and continue running that task.  These tasks typically have a repeating loop that continues to run until the Run Continuously Task is issued the `kill` command. |

### Run Continuously Task

We'll create a Run Continuously Task from our Orchestration.  This Task has an infinite `while` loop, so it will continue to loop until the Task is issued the `kill` command.

Run Continuously Tasks are generated Orchestration `run_continuously(name, json_args)` method.  Any arguments for the orchestration are passed in through the `json_args` parameter as a `Dict`.  If there are no arguments, then an empty set `{}` is passed.

In [176]:
# Example: run continuously

import datetime
task_start = datetime.datetime.now()
task_continuously = orchestration.run_continuously(name="run continuous sample", 
                              json_args={"workspace_name": workspace_name, 
                                         "pipeline_name": pipeline_name,
                                         "connection_name": connection_name
                                            })

### Task Status

The list of tasks in the Wallaroo instance is retrieves through the Wallaroo Client `list_tasks()` method.  This returns an array list of the following.

| Parameter | Type | Description |
| --- | --- | ---|
| **id** | string | The UUID identifier for the task. |
| **last run status** | string | The last reported status the task.  Values are: <br><ul><li>`pending`: The task has not been started.</li><li>`started`: The task has been scheduled to execute.</li><li>`pending_kill`: The task kill command has been issued and the task is scheduled to be stopped.</li></ul> |
| **type** | string | The type of the task.  Values are: <br><ul><li>`Temporary Run`: The task runs once then stop.</li><li>`Scheduled Run`: The task repeats on a `cron` like schedule.</li></ul> |
| **created at** | DateTime | The date and time the task was started. |
| **updated at** | DateTime | The date and time the task was updated. |

For this example, the status of the previously created task will be generated, then looped until it has reached status `started`.

In [177]:
while task_continuously.status() != "started":
    display(task_continuously.status())
    time.sleep(5)

### Task Results

We can view the inferences from our logs and verify that new entries were added from our task.  We can do that with the task `last_runs()` method to see the list of task runs executed, then show the log from the last completed task run.

In [178]:
# give time for the first task run to generate
time.sleep(30)
task_continuously.last_runs()

task id,pod id,status,created at,updated at
51ade31c-d69f-4f95-9b7a-b48096ef3d3b,0f8c2940-f025-4abc-9daa-b608e4dbde05,running,2024-24-Sep 20:17:03,2024-24-Sep 20:17:03


For this example, we'll update the Task Run logs ever 20 seconds to show the loop is executing.

In [179]:
for x in range (10):  
    time.sleep(20)
    display(task_continuously.last_runs()[0].logs())

[]

[]

['2024-09-24T20:17:10.86879146Z stdout F Getting the workspace continuoustutorial',
 '2024-09-24T20:17:10.918647416Z stdout F Getting the pipeline continuouspipeline',
 '2024-09-24T20:17:11.76110169Z stdout F Deploy the pipeline.',
 '2024-09-24T20:17:11.76114773Z stdout F Deployment initiated for continuouspipeline. Please check pipeline status.',
 "2024-09-24T20:17:11.76115947Z stdout F {'status': 'Running', 'details': [], 'engines': [{'ip': '10.28.2.58', 'name': 'engine-77cb97f568-lpgk4', 'status': 'Running', 'reason': None, 'details': [], 'pipeline_statuses': {'pipelines': [{'id': 'continuouspipeline', 'status': 'Running', 'version': '0bb33df8-532b-46f6-990f-5bc3e69dd04e'}]}, 'model_statuses': {'models': [{'model_version_id': 7, 'name': 'houseprice', 'sha': 'e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6', 'status': 'Running', 'version': '5b7438ae-04f0-4398-8dae-3d7a0370e746'}]}}], 'engine_lbs': [{'ip': '10.28.2.57', 'name': 'engine-lb-6b59985857-tkx7c', 'status': 'Running', 'reason': None, 'details': []}], 'sidekicks': []}",
 '2024-09-24T20:17:11.879440891Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:17:11.76116851Z stdout F Getting arrow table file',
 '2024-09-24T20:17:11.930446327Z stdout F 0 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930454137Z stdout F 1 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930407817Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:17:11.930460097Z stdout F 2 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930465247Z stdout F 3 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930473117Z stdout F ',
 '2024-09-24T20:17:11.930469197Z stdout F 4 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930477527Z stdout F [5 rows x 4 columns]',
 '2024-09-24T20:18:11.951983346Z stdout F Getting arrow table file',
 '2024-09-24T20:18:12.284001989Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:18:12.369996093Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:18:12.370047903Z stdout F 0 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370059403Z stdout F 1 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370069133Z stdout F 2 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370077063Z stdout F 3 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370084763Z stdout F 4 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370103233Z stdout F [5 rows x 4 columns]',
 '2024-09-24T20:18:12.370093593Z stdout F ']

['2024-09-24T20:17:10.86879146Z stdout F Getting the workspace continuoustutorial',
 '2024-09-24T20:17:10.918647416Z stdout F Getting the pipeline continuouspipeline',
 '2024-09-24T20:17:11.76110169Z stdout F Deploy the pipeline.',
 '2024-09-24T20:17:11.76114773Z stdout F Deployment initiated for continuouspipeline. Please check pipeline status.',
 "2024-09-24T20:17:11.76115947Z stdout F {'status': 'Running', 'details': [], 'engines': [{'ip': '10.28.2.58', 'name': 'engine-77cb97f568-lpgk4', 'status': 'Running', 'reason': None, 'details': [], 'pipeline_statuses': {'pipelines': [{'id': 'continuouspipeline', 'status': 'Running', 'version': '0bb33df8-532b-46f6-990f-5bc3e69dd04e'}]}, 'model_statuses': {'models': [{'model_version_id': 7, 'name': 'houseprice', 'sha': 'e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6', 'status': 'Running', 'version': '5b7438ae-04f0-4398-8dae-3d7a0370e746'}]}}], 'engine_lbs': [{'ip': '10.28.2.57', 'name': 'engine-lb-6b59985857-tkx7c', 'status': 'Running', 'reason': None, 'details': []}], 'sidekicks': []}",
 '2024-09-24T20:17:11.879440891Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:17:11.76116851Z stdout F Getting arrow table file',
 '2024-09-24T20:17:11.930446327Z stdout F 0 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930454137Z stdout F 1 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930407817Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:17:11.930460097Z stdout F 2 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930465247Z stdout F 3 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930473117Z stdout F ',
 '2024-09-24T20:17:11.930469197Z stdout F 4 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930477527Z stdout F [5 rows x 4 columns]',
 '2024-09-24T20:18:11.951983346Z stdout F Getting arrow table file',
 '2024-09-24T20:18:12.284001989Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:18:12.369996093Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:18:12.370047903Z stdout F 0 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370059403Z stdout F 1 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370069133Z stdout F 2 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370077063Z stdout F 3 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370084763Z stdout F 4 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370103233Z stdout F [5 rows x 4 columns]',
 '2024-09-24T20:18:12.370093593Z stdout F ']

['2024-09-24T20:17:10.86879146Z stdout F Getting the workspace continuoustutorial',
 '2024-09-24T20:17:10.918647416Z stdout F Getting the pipeline continuouspipeline',
 '2024-09-24T20:17:11.76110169Z stdout F Deploy the pipeline.',
 '2024-09-24T20:17:11.76114773Z stdout F Deployment initiated for continuouspipeline. Please check pipeline status.',
 "2024-09-24T20:17:11.76115947Z stdout F {'status': 'Running', 'details': [], 'engines': [{'ip': '10.28.2.58', 'name': 'engine-77cb97f568-lpgk4', 'status': 'Running', 'reason': None, 'details': [], 'pipeline_statuses': {'pipelines': [{'id': 'continuouspipeline', 'status': 'Running', 'version': '0bb33df8-532b-46f6-990f-5bc3e69dd04e'}]}, 'model_statuses': {'models': [{'model_version_id': 7, 'name': 'houseprice', 'sha': 'e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6', 'status': 'Running', 'version': '5b7438ae-04f0-4398-8dae-3d7a0370e746'}]}}], 'engine_lbs': [{'ip': '10.28.2.57', 'name': 'engine-lb-6b59985857-tkx7c', 'status': 'Running', 'reason': None, 'details': []}], 'sidekicks': []}",
 '2024-09-24T20:17:11.879440891Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:17:11.76116851Z stdout F Getting arrow table file',
 '2024-09-24T20:17:11.930446327Z stdout F 0 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930454137Z stdout F 1 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930407817Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:17:11.930460097Z stdout F 2 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930465247Z stdout F 3 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930473117Z stdout F ',
 '2024-09-24T20:17:11.930469197Z stdout F 4 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930477527Z stdout F [5 rows x 4 columns]',
 '2024-09-24T20:18:11.951983346Z stdout F Getting arrow table file',
 '2024-09-24T20:18:12.284001989Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:18:12.369996093Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:18:12.370047903Z stdout F 0 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370059403Z stdout F 1 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370069133Z stdout F 2 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370077063Z stdout F 3 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370084763Z stdout F 4 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370103233Z stdout F [5 rows x 4 columns]',
 '2024-09-24T20:18:12.370093593Z stdout F ']

['2024-09-24T20:17:10.86879146Z stdout F Getting the workspace continuoustutorial',
 '2024-09-24T20:17:10.918647416Z stdout F Getting the pipeline continuouspipeline',
 '2024-09-24T20:17:11.76110169Z stdout F Deploy the pipeline.',
 '2024-09-24T20:17:11.76114773Z stdout F Deployment initiated for continuouspipeline. Please check pipeline status.',
 "2024-09-24T20:17:11.76115947Z stdout F {'status': 'Running', 'details': [], 'engines': [{'ip': '10.28.2.58', 'name': 'engine-77cb97f568-lpgk4', 'status': 'Running', 'reason': None, 'details': [], 'pipeline_statuses': {'pipelines': [{'id': 'continuouspipeline', 'status': 'Running', 'version': '0bb33df8-532b-46f6-990f-5bc3e69dd04e'}]}, 'model_statuses': {'models': [{'model_version_id': 7, 'name': 'houseprice', 'sha': 'e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6', 'status': 'Running', 'version': '5b7438ae-04f0-4398-8dae-3d7a0370e746'}]}}], 'engine_lbs': [{'ip': '10.28.2.57', 'name': 'engine-lb-6b59985857-tkx7c', 'status': 'Running', 'reason': None, 'details': []}], 'sidekicks': []}",
 '2024-09-24T20:17:11.879440891Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:17:11.76116851Z stdout F Getting arrow table file',
 '2024-09-24T20:17:11.930407817Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:17:11.930454137Z stdout F 1 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930446327Z stdout F 0 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930465247Z stdout F 3 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930460097Z stdout F 2 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930469197Z stdout F 4 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930473117Z stdout F ',
 '2024-09-24T20:17:11.930477527Z stdout F [5 rows x 4 columns]',
 '2024-09-24T20:18:11.951983346Z stdout F Getting arrow table file',
 '2024-09-24T20:18:12.284001989Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:18:12.369996093Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:18:12.370047903Z stdout F 0 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370059403Z stdout F 1 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370069133Z stdout F 2 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370077063Z stdout F 3 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370084763Z stdout F 4 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370103233Z stdout F [5 rows x 4 columns]',
 '2024-09-24T20:18:12.370093593Z stdout F ',
 '2024-09-24T20:19:12.429861463Z stdout F Getting arrow table file',
 '2024-09-24T20:19:12.666226965Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:19:12.768620646Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:19:12.768700546Z stdout F 0 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768713437Z stdout F 1 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768722206Z stdout F 2 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768730846Z stdout F 3 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768738877Z stdout F 4 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768747917Z stdout F ',
 '2024-09-24T20:19:12.768758377Z stdout F [5 rows x 4 columns]']

['2024-09-24T20:17:10.86879146Z stdout F Getting the workspace continuoustutorial',
 '2024-09-24T20:17:10.918647416Z stdout F Getting the pipeline continuouspipeline',
 '2024-09-24T20:17:11.76110169Z stdout F Deploy the pipeline.',
 '2024-09-24T20:17:11.76114773Z stdout F Deployment initiated for continuouspipeline. Please check pipeline status.',
 "2024-09-24T20:17:11.76115947Z stdout F {'status': 'Running', 'details': [], 'engines': [{'ip': '10.28.2.58', 'name': 'engine-77cb97f568-lpgk4', 'status': 'Running', 'reason': None, 'details': [], 'pipeline_statuses': {'pipelines': [{'id': 'continuouspipeline', 'status': 'Running', 'version': '0bb33df8-532b-46f6-990f-5bc3e69dd04e'}]}, 'model_statuses': {'models': [{'model_version_id': 7, 'name': 'houseprice', 'sha': 'e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6', 'status': 'Running', 'version': '5b7438ae-04f0-4398-8dae-3d7a0370e746'}]}}], 'engine_lbs': [{'ip': '10.28.2.57', 'name': 'engine-lb-6b59985857-tkx7c', 'status': 'Running', 'reason': None, 'details': []}], 'sidekicks': []}",
 '2024-09-24T20:17:11.879440891Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:17:11.76116851Z stdout F Getting arrow table file',
 '2024-09-24T20:17:11.930407817Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:17:11.930454137Z stdout F 1 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930446327Z stdout F 0 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930465247Z stdout F 3 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930460097Z stdout F 2 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930469197Z stdout F 4 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930473117Z stdout F ',
 '2024-09-24T20:17:11.930477527Z stdout F [5 rows x 4 columns]',
 '2024-09-24T20:18:11.951983346Z stdout F Getting arrow table file',
 '2024-09-24T20:18:12.284001989Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:18:12.369996093Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:18:12.370047903Z stdout F 0 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370059403Z stdout F 1 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370069133Z stdout F 2 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370077063Z stdout F 3 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370084763Z stdout F 4 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370103233Z stdout F [5 rows x 4 columns]',
 '2024-09-24T20:18:12.370093593Z stdout F ',
 '2024-09-24T20:19:12.429861463Z stdout F Getting arrow table file',
 '2024-09-24T20:19:12.666226965Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:19:12.768620646Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:19:12.768700546Z stdout F 0 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768713437Z stdout F 1 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768722206Z stdout F 2 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768730846Z stdout F 3 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768738877Z stdout F 4 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768747917Z stdout F ',
 '2024-09-24T20:19:12.768758377Z stdout F [5 rows x 4 columns]']

['2024-09-24T20:17:10.86879146Z stdout F Getting the workspace continuoustutorial',
 '2024-09-24T20:17:10.918647416Z stdout F Getting the pipeline continuouspipeline',
 '2024-09-24T20:17:11.76110169Z stdout F Deploy the pipeline.',
 '2024-09-24T20:17:11.76114773Z stdout F Deployment initiated for continuouspipeline. Please check pipeline status.',
 "2024-09-24T20:17:11.76115947Z stdout F {'status': 'Running', 'details': [], 'engines': [{'ip': '10.28.2.58', 'name': 'engine-77cb97f568-lpgk4', 'status': 'Running', 'reason': None, 'details': [], 'pipeline_statuses': {'pipelines': [{'id': 'continuouspipeline', 'status': 'Running', 'version': '0bb33df8-532b-46f6-990f-5bc3e69dd04e'}]}, 'model_statuses': {'models': [{'model_version_id': 7, 'name': 'houseprice', 'sha': 'e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6', 'status': 'Running', 'version': '5b7438ae-04f0-4398-8dae-3d7a0370e746'}]}}], 'engine_lbs': [{'ip': '10.28.2.57', 'name': 'engine-lb-6b59985857-tkx7c', 'status': 'Running', 'reason': None, 'details': []}], 'sidekicks': []}",
 '2024-09-24T20:17:11.879440891Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:17:11.76116851Z stdout F Getting arrow table file',
 '2024-09-24T20:17:11.930407817Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:17:11.930454137Z stdout F 1 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930446327Z stdout F 0 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930465247Z stdout F 3 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930460097Z stdout F 2 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930469197Z stdout F 4 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930473117Z stdout F ',
 '2024-09-24T20:17:11.930477527Z stdout F [5 rows x 4 columns]',
 '2024-09-24T20:18:11.951983346Z stdout F Getting arrow table file',
 '2024-09-24T20:18:12.284001989Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:18:12.369996093Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:18:12.370047903Z stdout F 0 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370059403Z stdout F 1 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370069133Z stdout F 2 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370077063Z stdout F 3 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370084763Z stdout F 4 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370103233Z stdout F [5 rows x 4 columns]',
 '2024-09-24T20:18:12.370093593Z stdout F ',
 '2024-09-24T20:19:12.429861463Z stdout F Getting arrow table file',
 '2024-09-24T20:19:12.666226965Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:19:12.768620646Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:19:12.768700546Z stdout F 0 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768713437Z stdout F 1 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768722206Z stdout F 2 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768730846Z stdout F 3 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768738877Z stdout F 4 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768747917Z stdout F ',
 '2024-09-24T20:19:12.768758377Z stdout F [5 rows x 4 columns]']

['2024-09-24T20:17:10.86879146Z stdout F Getting the workspace continuoustutorial',
 '2024-09-24T20:17:10.918647416Z stdout F Getting the pipeline continuouspipeline',
 '2024-09-24T20:17:11.76110169Z stdout F Deploy the pipeline.',
 '2024-09-24T20:17:11.76114773Z stdout F Deployment initiated for continuouspipeline. Please check pipeline status.',
 "2024-09-24T20:17:11.76115947Z stdout F {'status': 'Running', 'details': [], 'engines': [{'ip': '10.28.2.58', 'name': 'engine-77cb97f568-lpgk4', 'status': 'Running', 'reason': None, 'details': [], 'pipeline_statuses': {'pipelines': [{'id': 'continuouspipeline', 'status': 'Running', 'version': '0bb33df8-532b-46f6-990f-5bc3e69dd04e'}]}, 'model_statuses': {'models': [{'model_version_id': 7, 'name': 'houseprice', 'sha': 'e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6', 'status': 'Running', 'version': '5b7438ae-04f0-4398-8dae-3d7a0370e746'}]}}], 'engine_lbs': [{'ip': '10.28.2.57', 'name': 'engine-lb-6b59985857-tkx7c', 'status': 'Running', 'reason': None, 'details': []}], 'sidekicks': []}",
 '2024-09-24T20:17:11.879440891Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:17:11.76116851Z stdout F Getting arrow table file',
 '2024-09-24T20:17:11.930407817Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:17:11.930454137Z stdout F 1 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930446327Z stdout F 0 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930460097Z stdout F 2 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930465247Z stdout F 3 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930469197Z stdout F 4 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930473117Z stdout F ',
 '2024-09-24T20:17:11.930477527Z stdout F [5 rows x 4 columns]',
 '2024-09-24T20:18:11.951983346Z stdout F Getting arrow table file',
 '2024-09-24T20:18:12.284001989Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:18:12.369996093Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:18:12.370047903Z stdout F 0 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370059403Z stdout F 1 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370069133Z stdout F 2 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370077063Z stdout F 3 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370084763Z stdout F 4 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370093593Z stdout F ',
 '2024-09-24T20:18:12.370103233Z stdout F [5 rows x 4 columns]',
 '2024-09-24T20:19:12.429861463Z stdout F Getting arrow table file',
 '2024-09-24T20:19:12.666226965Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:19:12.768620646Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:19:12.768700546Z stdout F 0 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768713437Z stdout F 1 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768722206Z stdout F 2 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768730846Z stdout F 3 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768738877Z stdout F 4 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768747917Z stdout F ',
 '2024-09-24T20:19:12.768758377Z stdout F [5 rows x 4 columns]',
 '2024-09-24T20:20:12.816141828Z stdout F Getting arrow table file',
 '2024-09-24T20:20:13.049633599Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:20:13.136137823Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:20:13.136203553Z stdout F 0 2024-09-24 20:20:13.119  ...             0',
 '2024-09-24T20:20:13.136212942Z stdout F 1 2024-09-24 20:20:13.119  ...             0',
 '2024-09-24T20:20:13.136219102Z stdout F 2 2024-09-24 20:20:13.119  ...             0',
 '2024-09-24T20:20:13.1362

['2024-09-24T20:17:10.86879146Z stdout F Getting the workspace continuoustutorial',
 '2024-09-24T20:17:10.918647416Z stdout F Getting the pipeline continuouspipeline',
 '2024-09-24T20:17:11.76110169Z stdout F Deploy the pipeline.',
 '2024-09-24T20:17:11.76114773Z stdout F Deployment initiated for continuouspipeline. Please check pipeline status.',
 "2024-09-24T20:17:11.76115947Z stdout F {'status': 'Running', 'details': [], 'engines': [{'ip': '10.28.2.58', 'name': 'engine-77cb97f568-lpgk4', 'status': 'Running', 'reason': None, 'details': [], 'pipeline_statuses': {'pipelines': [{'id': 'continuouspipeline', 'status': 'Running', 'version': '0bb33df8-532b-46f6-990f-5bc3e69dd04e'}]}, 'model_statuses': {'models': [{'model_version_id': 7, 'name': 'houseprice', 'sha': 'e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6', 'status': 'Running', 'version': '5b7438ae-04f0-4398-8dae-3d7a0370e746'}]}}], 'engine_lbs': [{'ip': '10.28.2.57', 'name': 'engine-lb-6b59985857-tkx7c', 'status': 'Running', 'reason': None, 'details': []}], 'sidekicks': []}",
 '2024-09-24T20:17:11.879440891Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:17:11.76116851Z stdout F Getting arrow table file',
 '2024-09-24T20:17:11.930407817Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:17:11.930454137Z stdout F 1 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930446327Z stdout F 0 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930460097Z stdout F 2 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930465247Z stdout F 3 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930469197Z stdout F 4 2024-09-24 20:17:11.906  ...             0',
 '2024-09-24T20:17:11.930473117Z stdout F ',
 '2024-09-24T20:17:11.930477527Z stdout F [5 rows x 4 columns]',
 '2024-09-24T20:18:11.951983346Z stdout F Getting arrow table file',
 '2024-09-24T20:18:12.284001989Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:18:12.369996093Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:18:12.370047903Z stdout F 0 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370059403Z stdout F 1 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370069133Z stdout F 2 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370077063Z stdout F 3 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370084763Z stdout F 4 2024-09-24 20:18:12.353  ...             0',
 '2024-09-24T20:18:12.370093593Z stdout F ',
 '2024-09-24T20:18:12.370103233Z stdout F [5 rows x 4 columns]',
 '2024-09-24T20:19:12.429861463Z stdout F Getting arrow table file',
 '2024-09-24T20:19:12.666226965Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:19:12.768620646Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:19:12.768700546Z stdout F 0 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768713437Z stdout F 1 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768722206Z stdout F 2 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768730846Z stdout F 3 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768738877Z stdout F 4 2024-09-24 20:19:12.748  ...             0',
 '2024-09-24T20:19:12.768747917Z stdout F ',
 '2024-09-24T20:19:12.768758377Z stdout F [5 rows x 4 columns]',
 '2024-09-24T20:20:12.816141828Z stdout F Getting arrow table file',
 '2024-09-24T20:20:13.049633599Z stdout F Inference time.  Displaying results after.',
 '2024-09-24T20:20:13.136137823Z stdout F                      time  ... anomaly.count',
 '2024-09-24T20:20:13.136203553Z stdout F 0 2024-09-24 20:20:13.119  ...             0',
 '2024-09-24T20:20:13.136212942Z stdout F 1 2024-09-24 20:20:13.119  ...             0',
 '2024-09-24T20:20:13.136219102Z stdout F 2 2024-09-24 20:20:13.119  ...             0',
 '2024-09-24T20:20:13.1362

### Kill Task

With our example complete, we will kill the task and undeploy the pipeline.

In [180]:
task_continuously.kill()

<ArbexStatus.PENDING_KILL: 'pending_kill'>

In [181]:
pipeline.undeploy()

Waiting for undeployment - this will take up to 45s .................................... ok


name,continuouspipeline
created,2024-09-24 17:11:25.717188+00:00
last_updated,2024-09-24 17:57:40.189304+00:00
deployed,False
workspace_id,9
workspace_name,continuoustutorial
arch,x86
accel,none
tags,
versions,"ba43237a-94ca-4375-a259-484775e9ed46, fd88bf40-1305-491f-9c93-7c27fc49e754, bf8e8426-d3ae-4f8d-a549-3ebd7bb30ae8, 6be2d1d9-719f-42ff-8cd6-d253ad924b7f, 759bd273-6b20-4cc2-87c1-7c48b81686f4, 085f7dd4-72fb-4a16-928b-bf3fb71b0baa, 8ae1c528-d714-4060-af99-98dbf0a93dd0"
steps,houseprice
